In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import numpy as np

%matplotlib inline

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
tf.contrib.layers.fully_connected??

In [8]:
class ActorCriticNetwork:
    def __init__(self):
        """
            1) fully connected NN to encode board state into 
            higher dimensional features. input_size = 17 (flattened board)
            
            2) LSTM takes previous output and outputs a new output
            
            3) num_outputs = action_space (7 possible actions)
            
            4) Initialize fully connected network for policy(softmax)
            and value(regressor) functions
            
            5) Initialize weights for each layer
            
            6) train model
        """
    def __call__(self, input_game_state):
        """
            // Move
            1) take the input_game_state and forward propagate it through pipeline
            2) Softmax over result from pipeline
            3) select valid action by sampling across the action distribution
            generated by the network
        """
    def fit(self):
         """
        1) Initialize model
        
        2) select optimizer
        
        3) for each epoch
             sync local params with global model
             if just starting initialize params (c and h)
             
             collect values, log probabilities, rewards and entropies
             for steps in each epoch:
                pass game state to model and get logits and value
                pass logits to softmax and log_softmax to get log_prob and prob
                calculate entropy
                add to entropy list
                
                calculate action from probs
                calculate the lob prob given action
                
                execute action on game to get reward, next_state, 
                and boolean that describes if game over
                
                add new values to the values list
                add log_probs to log_probs list
                add rewards to rewards list
                
            initialize rewards as 0
            
            add rewards to values list
            iterate over all generated rewards(reverse order)
                # calculate advantage given discounted reward
                # calculate policy_loss and value_loss given advantages
                R = args.gamma * R + rewards[i]
                advantage = R - values[i]
                value_loss = value_loss + 0.5 * advantage.pow(2)

                # Generalized Advantage Estimataion
                delta_t = rewards[i] + args.gamma * \
                    values[i + 1].data - values[i].data
                gae = gae * args.gamma * args.tau + delta_t

                policy_loss = policy_loss - \
                    log_probs[i] * Variable(gae) - args.beta * entropies[i]
            
            update params given gradients of losses
            push params up to the global model
    """
        
class Agent:
    def __init__(self):
        """
            1) seed (Random nums)
            2) game environment
            3) Agent has ActorCriticNetwork( initial_state, )
            

            
        """
        
class TrainParameters:
    """
        1) loads any existing weights
        2) Spawns separate worker copies of ActorCriticModel
    """